# Generate top of the book & trades using dedicated service - C#

### Overview
Retrieving trades and top of the book data points over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request simultaneously trades and top of the book and illustrate market activity over a look back period.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades and best bid/ask over the time.

### Inputs/outputs
Trades and best Bid/Ask extraction sample requires instrument's identifier, date time intervals as per inputs and returns top fo the book the occured trades data points.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and top of the book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and top of the book tick data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V2*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***

# Run Top of the book & Trades sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

using TickTradesAndBookService = Systemathics.Apis.Services.Tick.V2.TickTradesAndBookService;
using TickTradesAndBookRequest = Systemathics.Apis.Services.Tick.V2.TickTradesAndBookRequest;
using TickTradesAndBookResponse = Systemathics.Apis.Services.Tick.V2.TickTradesAndBookResponse;
using Trade = Systemathics.Apis.Type.Shared.V2.Trade;
using Book = Systemathics.Apis.Type.Shared.V2.Book;
using BookUpdates = Systemathics.Apis.Type.Shared.V2.BookUpdates;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = new Grpc.Core.Metadata();
try { headers = TokenHelpers.GetTokenAsMetaData(); } catch {}

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";
// var ticker = "E:BNP";
// var exchange = "EQUITY_L2_973";

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
var start = new DateTime(2021, 11, 01);
var end = new DateTime(2021, 11, 01);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
var timeInterval = new TimeInterval()
{
     StartTime = new TimeOfDay { Hours = 14, Minutes = 00, Seconds = 00 },
     EndTime = new TimeOfDay { Hours = 14, Minutes = 30, Seconds = 00 } 
};

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

The following code snippets create requests for *trades* and *top of the book* data and instantiates the service:

In [ ]:
// Generate the tick trades and book request
var request = new TickTradesAndBookRequest{ 
    Constraints = constraints, 
    MaxDepth = 1, 
    BookUpdates = BookUpdates.SnapshotsOnly,
    Adjustment = false
};
var identifier = new IdentifierAndLevel { Exchange = exchange, Ticker = ticker, Level = Level.TradesAndBook };
request.Identifiers.Add(new [] {identifier}); 

In [ ]:
// Instantiate the tick trades and book service
var service = new TickTradesAndBookService.TickTradesAndBookServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve top of the book & trades

In [ ]:
// Get the trades and the book limits
var trades = new List<Tuple<Timestamp, Trade>>();
var books = new List<Tuple<Timestamp, Book>>();

var call = service.TickTradesAndBook(request, headers);
await foreach (var current in call.ResponseStream.ReadAllAsync())
{
    if(current.Mapping != null)
    {
        // Skip the mapping data
        continue;
    }

    var ts = current.Data.TimeStamp;

    // Trade
    if (current.Data.Trade != null) 
    {      
        trades.Add(new Tuple<Timestamp, Trade>(ts, current.Data.Trade));
    }
    
    // Book
    if (current.Data.Book != null) 
    {      
        books.Add(new Tuple<Timestamp, Book>(ts, current.Data.Book));
    }
}
// Display data
// display(trades);
// display(books);

#### 4.2 Plot top of the book & trade prices
Display *best bid*, *best ask* and *trade prices* over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graph with the prices as follows: 

In [ ]:
// Plot simultaneously Bid, Ask and trade prices
var bidAskTradePrices = new List<Scattergl>
{
    new Scattergl
    {
        name = "AskPrice",
        mode = "lines",
        x = books.Select(elt => elt.Item1.ToDateTime()),
        y = books.Select(elt => elt.Item2.Ask[0].Price)
    },
    new Scattergl
    {
        name = "BidPrice",
        mode = "lines",
        x = books.Select(elt => elt.Item1.ToDateTime()),
        y = books.Select(elt => elt.Item2.Bid[0].Price)
    },
    new Scattergl
    {
        name = "TradePrice",
        mode = "lines",
        x = trades.Select(elt => elt.Item1.ToDateTime()),
        y = trades.Select(elt => elt.Item2.Price)
    }
};
var chart = Chart.Plot(bidAskTradePrices);
chart.WithTitle($"Bid/Ask and trade prices for {ticker}");
chart.WithXTitle($"Time");
chart.WithYTitle($"Price");
chart.Width = 1500;
chart.Height = 800;
display(chart);

In [ ]:
// Plot simultaneously Bid, Ask and trade sizes
var bidAskTradeSizes = new List<Scattergl>
{
    new Scattergl
    {
        name = "AskSize",
        mode = "lines",
        x = books.Select(elt => elt.Item1.ToDateTime()),
        y = books.Select(elt => -elt.Item2.Ask[0].Size)
    },
    new Scattergl
    {
        name = "BidSize",
        mode = "lines",
        x = books.Select(elt => elt.Item1.ToDateTime()),
        y = books.Select(elt => elt.Item2.Bid[0].Size)
    },
    new Scattergl
    {
        name = "TradeSize",
        mode = "markers",
        x = trades.Select(elt => elt.Item1.ToDateTime()),
        y = trades.Select(elt => elt.Item2.Size)
    }
};
var chart = Chart.Plot(bidAskTradeSizes);
chart.WithTitle($"Bid/Ask and trade sizes for {ticker}");
chart.WithXTitle($"Time");
chart.WithYTitle($"Size");
chart.Width = 1500;
chart.Height = 800;
display(chart);